In [ ]:
pip install pyspark

In [ ]:
from pyspark.sql import SparkSession
#initialize sparkSession
spark =  SparkSession.builder\
    .appName("Pyspark notebook example")\
    .getOrCreate()

print(spark)

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
#Initialize SparkSession
spark = SparkSession.builder \
.appName("PySpark DataFrame Example") \
.getOrCreate()
#Sample data representing employees
data = [
("John Doe", "Engineering", 75000),
("Jane Smith", "Marketing", 60000),
("Sam Brown", "Engineering", 80000),
("Emily Davis", "HR", 50000),
("Michael Johnson", "Marketing", 70000),
]
#Define schema for DataFrame
columns= ["Name", "Department", "Salary"]
#Create DataFrame
df = spark.createDataFrame(data, schema=columns)
#Show the DataFrame
df.show()

+---------------+-----------+------+
|           Name| Department|Salary|
+---------------+-----------+------+
|       John Doe|Engineering| 75000|
|     Jane Smith|  Marketing| 60000|
|      Sam Brown|Engineering| 80000|
|    Emily Davis|         HR| 50000|
|Michael Johnson|  Marketing| 70000|
+---------------+-----------+------+



In [ ]:

# Filter: Select employees with a salary greater than 65,000
high_salary_df = df.filter (col ("Salary") > 65000)
print("Employees with Salary > 65,000:")
high_salary_df.show()

Employees with Salary > 65,000:
+---------------+-----------+------+
|           Name| Department|Salary|
+---------------+-----------+------+
|       John Doe|Engineering| 75000|
|      Sam Brown|Engineering| 80000|
|Michael Johnson|  Marketing| 70000|
+---------------+-----------+------+



In [ ]:
# Group by Department and calculate the average salary
avg_salary_df = df.groupBy("Department").avg("Salary")
print("Average Salary by Department:")
avg_salary_df.show()

Average Salary by Department:
+-----------+-----------+
| Department|avg(Salary)|
+-----------+-----------+
|Engineering|    77500.0|
|  Marketing|    65000.0|
|         HR|    50000.0|
+-----------+-----------+



In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
#Initialize SparkSession
spark = SparkSession.builder\
.appName("Customer Transactions Analysis") \
.getOrCreate ()
# Sample data for customers
customers = [
(1, "Ravi", "Mumbai"),
(2, "Priya", "Delhi"),
(3, "Vijay", "Bangalore"),
(4, "Anita", "Chennai"),
(5, "Raj", "Hyderabad"),
]
# Sample data for transactions
transactions = [
(1, 1, 10000.50),
(2, 2, 20000.75),
(3, 1, 15000.25),
(4, 3, 30000.00),
(5, 2, 40000.50),
(6, 4, 25000.00),
(7, 5, 18000.75),
(8, 1, 5000.00),
]

#Define schema for DataFrames
customer_columns = ["CustomerID", "Name", "City"]
transaction_columns = ["TransactionID", "CustomerID", "Amount"]
#Create DataFrames
customer_df = spark.createDataFrame (customers, schema=customer_columns)
transaction_df = spark.createDataFrame (transactions, schema=transaction_columns)

#Show the DataFrames
print("Customers DataFrame:")
customer_df.show()
print("Transactions DataFrame:")
transaction_df.show()

Customers DataFrame:
+----------+-----+---------+
|CustomerID| Name|     City|
+----------+-----+---------+
|         1| Ravi|   Mumbai|
|         2|Priya|    Delhi|
|         3|Vijay|Bangalore|
|         4|Anita|  Chennai|
|         5|  Raj|Hyderabad|
+----------+-----+---------+

Transactions DataFrame:
+-------------+----------+--------+
|TransactionID|CustomerID|  Amount|
+-------------+----------+--------+
|            1|         1| 10000.5|
|            2|         2|20000.75|
|            3|         1|15000.25|
|            4|         3| 30000.0|
|            5|         2| 40000.5|
|            6|         4| 25000.0|
|            7|         5|18000.75|
|            8|         1|  5000.0|
+-------------+----------+--------+



In [ ]:
#Join the DataFrames on CustomerID
customer_transactions_df = customer_df.join(transaction_df, on="CustomerID")
print("Customer Transactions DataFrame:")
customer_transactions_df.show()

#Calculate the total amount spent by each customer
total_spent_df = customer_transactions_df.groupBy("Name").sum("Amount").withColumnRenamed ("sum (Amount)", "TotalSpent")
print("Total Amount Spent by Each Customer:")
total_spent_df.show()

## Find customers who have spent more than ₹30,000
big_spenders_df = total_spent_df.filter (col ("TotalSpent") > 30000)
print("Customers Who Spent More Than ₹30,000:")
big_spenders_df.show()

## Count the number of transactions per customer
transactions_count_df = customer_transactions_df.groupBy("Name").count().withColumnRenamed ("count", "TransactionCount")
print("Number of Transactions Per Customer:")
transactions_count_df.show()

## Sort customers by total amount spent in descending order
sorted_spenders_df = total_spent_df.orderBy (col ("TotalSpent").desc())
print("Customers Sorted by Total Spent (Descending):")
sorted_spenders_df.show()

Customer Transactions DataFrame:
+----------+-----+---------+-------------+--------+
|CustomerID| Name|     City|TransactionID|  Amount|
+----------+-----+---------+-------------+--------+
|         1| Ravi|   Mumbai|            1| 10000.5|
|         1| Ravi|   Mumbai|            3|15000.25|
|         1| Ravi|   Mumbai|            8|  5000.0|
|         2|Priya|    Delhi|            2|20000.75|
|         2|Priya|    Delhi|            5| 40000.5|
|         3|Vijay|Bangalore|            4| 30000.0|
|         4|Anita|  Chennai|            6| 25000.0|
|         5|  Raj|Hyderabad|            7|18000.75|
+----------+-----+---------+-------------+--------+

Total Amount Spent by Each Customer:
+-----+-----------+
| Name|sum(Amount)|
+-----+-----------+
| Ravi|   30000.75|
|Priya|   60001.25|
|Vijay|    30000.0|
|Anita|    25000.0|
|  Raj|   18000.75|
+-----+-----------+



AnalysisException: [UNRESOLVED_COLUMN.WITH_SUGGESTION] A column or function parameter with name `TotalSpent` cannot be resolved. Did you mean one of the following? [`Name`, `sum(Amount)`].;
'Filter ('TotalSpent > 30000)
+- Aggregate [Name#55], [Name#55, sum(Amount#62) AS sum(Amount)#226]
   +- Project [CustomerID#54L, Name#55, City#56, TransactionID#60L, Amount#62]
      +- Join Inner, (CustomerID#54L = CustomerID#61L)
         :- LogicalRDD [CustomerID#54L, Name#55, City#56], false
         +- LogicalRDD [TransactionID#60L, CustomerID#61L, Amount#62], false


In [ ]:
#### **Exercise: Product Sales Analysis**
#### **Step 1: Create DataFrames**

from pyspark.sql import SparkSession
from pyspark.sql.functions import col

# Initialize SparkSession
spark = SparkSession.builder \
    .appName("Product Sales Analysis") \
    .getOrCreate()

# Sample data for products
products = [
    (1, "Laptop", "Electronics", 50000),
    (2, "Smartphone", "Electronics", 30000),
    (3, "Table", "Furniture", 15000),
    (4, "Chair", "Furniture", 5000),
    (5, "Headphones", "Electronics", 2000),
]

# Sample data for sales transactions
sales = [
    (1, 1, 2),
    (2, 2, 1),
    (3, 3, 3),
    (4, 1, 1),
    (5, 4, 5),
    (6, 2, 2),
    (7, 5, 10),
    (8, 3, 1),
]

#Define schema for DataFrames
product_columns = ["ProductID", "Name", "Category", "Price"]
sales_columns = ["SaleID", "ProductID", "Quantity"]
#Create DataFrames
product_df = spark.createDataFrame (products, schema=product_columns)
transaction_columns = ["TransactionID", "CustomerID", "Amount"]
#Create DataFrames
product_df = spark.createDataFrame (products, schema=product_columns)
sales_df = spark.createDataFrame (sales, schema=sales_columns)

#Show the DataFrames
print("Products DataFrame:")
product_df.show()
print("Sales DataFrame:")
sales_df.show()

Products DataFrame:
+---------+----------+-----------+-----+
|ProductID|      Name|   Category|Price|
+---------+----------+-----------+-----+
|        1|    Laptop|Electronics|50000|
|        2|Smartphone|Electronics|30000|
|        3|     Table|  Furniture|15000|
|        4|     Chair|  Furniture| 5000|
|        5|Headphones|Electronics| 2000|
+---------+----------+-----------+-----+

Sales DataFrame:
+------+---------+--------+
|SaleID|ProductID|Quantity|
+------+---------+--------+
|     1|        1|       2|
|     2|        2|       1|
|     3|        3|       3|
|     4|        1|       1|
|     5|        4|       5|
|     6|        2|       2|
|     7|        5|      10|
|     8|        3|       1|
+------+---------+--------+



In [ ]:
#### **Step 2: Perform the Following Tasks**
#1. **Join the DataFrames:**

product_sales_df = product_df.join(sales_df, on="ProductID")
print("Product Sales DataFrame:")
product_sales_df.show()

Product Sales DataFrame:
+---------+----------+-----------+-----+------+--------+
|ProductID|      Name|   Category|Price|SaleID|Quantity|
+---------+----------+-----------+-----+------+--------+
|        1|    Laptop|Electronics|50000|     1|       2|
|        1|    Laptop|Electronics|50000|     4|       1|
|        2|Smartphone|Electronics|30000|     2|       1|
|        2|Smartphone|Electronics|30000|     6|       2|
|        3|     Table|  Furniture|15000|     3|       3|
|        3|     Table|  Furniture|15000|     8|       1|
|        4|     Chair|  Furniture| 5000|     5|       5|
|        5|Headphones|Electronics| 2000|     7|      10|
+---------+----------+-----------+-----+------+--------+



In [ ]:
#2. **Calculate Total Sales Value:**
combined_df = product_sales_df.withColumn("TotalSalesValue", col("Price") * col("Quantity"))
print("Combined DataFrame with Total Sales Value:")
combined_df.show()

Combined DataFrame with Total Sales Value:
+---------+----------+-----------+-----+------+--------+---------------+
|ProductID|      Name|   Category|Price|SaleID|Quantity|TotalSalesValue|
+---------+----------+-----------+-----+------+--------+---------------+
|        1|    Laptop|Electronics|50000|     1|       2|         100000|
|        1|    Laptop|Electronics|50000|     4|       1|          50000|
|        2|Smartphone|Electronics|30000|     2|       1|          30000|
|        2|Smartphone|Electronics|30000|     6|       2|          60000|
|        3|     Table|  Furniture|15000|     3|       3|          45000|
|        3|     Table|  Furniture|15000|     8|       1|          15000|
|        4|     Chair|  Furniture| 5000|     5|       5|          25000|
|        5|Headphones|Electronics| 2000|     7|      10|          20000|
+---------+----------+-----------+-----+------+--------+---------------+



In [ ]:
#3. **Find the Total Sales for Each Product Category:**
category_sales_df = combined_df.groupBy("Category").agg({"TotalSalesValue": "sum"}).withColumnRenamed("sum(TotalSalesValue)", "TotalCategorySales")
print("Total Sales for Each Product Category:")
category_sales_df.show()

Total Sales for Each Product Category:
+-----------+------------------+
|   Category|TotalCategorySales|
+-----------+------------------+
|Electronics|            260000|
|  Furniture|             85000|
+-----------+------------------+



In [ ]:
#4. **Identify the Top-Selling Product:**
product_sales_df = combined_df.groupBy("ProductID", "Name").agg({"TotalSalesValue": "sum"}).withColumnRenamed("sum(TotalSalesValue)", "TotalProductSales")
top_selling_product_df = product_sales_df.orderBy(col("TotalProductSales").desc()).limit(1)
print("Top-Selling Product:")
top_selling_product_df.show()



Top-Selling Product:
+---------+------+-----------------+
|ProductID|  Name|TotalProductSales|
+---------+------+-----------------+
|        1|Laptop|           150000|
+---------+------+-----------------+



In [ ]:
#5. **Sort the Products by Total Sales Value:**
sorted_products_df = combined_df.orderBy(col("TotalSalesValue").desc())
print("Products Sorted by Total Sales Value:")
sorted_products_df.show()

Products Sorted by Total Sales Value:
+---------+----------+-----------+-----+------+--------+---------------+
|ProductID|      Name|   Category|Price|SaleID|Quantity|TotalSalesValue|
+---------+----------+-----------+-----+------+--------+---------------+
|        1|    Laptop|Electronics|50000|     1|       2|         100000|
|        2|Smartphone|Electronics|30000|     6|       2|          60000|
|        1|    Laptop|Electronics|50000|     4|       1|          50000|
|        3|     Table|  Furniture|15000|     3|       3|          45000|
|        2|Smartphone|Electronics|30000|     2|       1|          30000|
|        4|     Chair|  Furniture| 5000|     5|       5|          25000|
|        5|Headphones|Electronics| 2000|     7|      10|          20000|
|        3|     Table|  Furniture|15000|     8|       1|          15000|
+---------+----------+-----------+-----+------+--------+---------------+



In [ ]:
#6. **Count the Number of Sales for Each Product:**
sales_count_df = sales_df.groupBy("ProductID").count().withColumnRenamed("count", "NumberOfSales")
print("Number of Sales for Each Product:")
sales_count_df.show()


Number of Sales for Each Product:
+---------+-------------+
|ProductID|NumberOfSales|
+---------+-------------+
|        1|            2|
|        3|            2|
|        2|            2|
|        5|            1|
|        4|            1|
+---------+-------------+



In [ ]:
#7. **Filter the Products with Total Sales Value Greater Than ₹50,000:**
product_sales_df = product_sales_df.withColumnRenamed("sum(TotalSalesValue)", "TotalProductSales")
filtered_products_df = product_sales_df.filter(col("TotalProductSales") > 50000)
print("Products with Total Sales Value > ₹50,000:")
filtered_products_df.show()

Products with Total Sales Value > ₹50,000:
+---------+----------+-----------------+
|ProductID|      Name|TotalProductSales|
+---------+----------+-----------------+
|        1|    Laptop|           150000|
|        2|Smartphone|            90000|
|        3|     Table|            60000|
+---------+----------+-----------------+



In [46]:
### **Exercise: Analyzing a Sample Sales Dataset Using PySpark**
### **Part 1: Dataset Preparation**

#### **Step 1: Generate the Sample Sales Dataset**
import pandas as pd
from datetime import datetime

# Sample sales data
data = {
       "TransactionID": [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
       "CustomerID": [101, 102, 103, 101, 104, 102, 103, 104, 101, 105],
       "ProductID": [501, 502, 501, 503, 504, 502, 503, 504, 501, 505],
       "Quantity": [2, 1, 4, 3, 1, 2, 5, 1, 2, 1],
       "Price": [150.0, 250.0, 150.0, 300.0, 450.0, 250.0, 300.0, 450.0, 150.0, 550.0],
       "Date": [
           datetime(2024, 9, 1),
           datetime(2024, 9, 1),
           datetime(2024, 9, 2),
           datetime(2024, 9, 2),
           datetime(2024, 9, 3),
           datetime(2024, 9, 3),
           datetime(2024, 9, 4),
           datetime(2024, 9, 4),
           datetime(2024, 9, 5),
           datetime(2024, 9, 5)
       ]
   }

# Create a DataFrame
pandas_df = pd.DataFrame(data)

# Save the DataFrame to a CSV file
pandas_df.to_csv('sales_data.csv', index=False)

print("Sample sales dataset has been created and saved as 'sales_data.csv'.")

#2. **Verify the Dataset:**
pandas_df = pd.read_csv('sales_data.csv')
print(pandas_df)


Sample sales dataset has been created and saved as 'sales_data.csv'.
   TransactionID  CustomerID  ProductID  Quantity  Price        Date
0              1         101        501         2  150.0  2024-09-01
1              2         102        502         1  250.0  2024-09-01
2              3         103        501         4  150.0  2024-09-02
3              4         101        503         3  300.0  2024-09-02
4              5         104        504         1  450.0  2024-09-03
5              6         102        502         2  250.0  2024-09-03
6              7         103        503         5  300.0  2024-09-04
7              8         104        504         1  450.0  2024-09-04
8              9         101        501         2  150.0  2024-09-05
9             10         105        505         1  550.0  2024-09-05


In [47]:
#### **Step 2: Load the Dataset into PySpark**

#1. **Initialize the SparkSession:**
spark = SparkSession.builder \
    .appName("Sales Dataset Analysis") \
    .getOrCreate()

#2. **Load the CSV File into a PySpark DataFrame:**
df = spark.read.csv('sales_data.csv', header=True, inferSchema=True)
#Display the first few rows
df.show(5)

+-------------+----------+---------+--------+-----+----------+
|TransactionID|CustomerID|ProductID|Quantity|Price|      Date|
+-------------+----------+---------+--------+-----+----------+
|            1|       101|      501|       2|150.0|2024-09-01|
|            2|       102|      502|       1|250.0|2024-09-01|
|            3|       103|      501|       4|150.0|2024-09-02|
|            4|       101|      503|       3|300.0|2024-09-02|
|            5|       104|      504|       1|450.0|2024-09-03|
+-------------+----------+---------+--------+-----+----------+
only showing top 5 rows



In [48]:
#### **Step 3: Explore the Data**
#1. **Print the Schema:**
df.printSchema()

#2. **Show the First Few Rows:**
df.show(5)

#3. **Get Summary Statistics:**
df.describe("Quantity", "Price").show()

root
 |-- TransactionID: integer (nullable = true)
 |-- CustomerID: integer (nullable = true)
 |-- ProductID: integer (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- Price: double (nullable = true)
 |-- Date: date (nullable = true)

+-------------+----------+---------+--------+-----+----------+
|TransactionID|CustomerID|ProductID|Quantity|Price|      Date|
+-------------+----------+---------+--------+-----+----------+
|            1|       101|      501|       2|150.0|2024-09-01|
|            2|       102|      502|       1|250.0|2024-09-01|
|            3|       103|      501|       4|150.0|2024-09-02|
|            4|       101|      503|       3|300.0|2024-09-02|
|            5|       104|      504|       1|450.0|2024-09-03|
+-------------+----------+---------+--------+-----+----------+
only showing top 5 rows

+-------+-----------------+-----------------+
|summary|         Quantity|            Price|
+-------+-----------------+-----------------+
|  count|            

In [52]:
#### **Step 4: Perform Data Transformations and Analysis**
#1. **Calculate the Total Sales Value for Each Transaction:**
df = df.withColumn("TotalSales", col("Quantity") * col("Price"))
df.show()

#2. **Group By ProductID and Calculate Total Sales Per Product:**
df.groupBy("ProductID").sum("TotalSales").alias("TotalProductSales").show()

#3. **Identify the Top-Selling Product:**
from pyspark.sql.functions import desc
df.groupBy("ProductID").sum("TotalSales").alias("TotalProductSales").orderBy(desc("sum(TotalSales)")).show(1)

#4. **Calculate the Total Sales by Date:**
df.groupBy("Date").sum("TotalSales").alias("TotalSalesByDate").orderBy("Date").show()

#5. **Filter High-Value Transactions:**
df.filter(col("TotalSales") > 500).show()


+-------------+----------+---------+--------+-----+----------+---------------+----------+
|TransactionID|CustomerID|ProductID|Quantity|Price|      Date|TotalSalesValue|TotalSales|
+-------------+----------+---------+--------+-----+----------+---------------+----------+
|            1|       101|      501|       2|150.0|2024-09-01|          300.0|     300.0|
|            2|       102|      502|       1|250.0|2024-09-01|          250.0|     250.0|
|            3|       103|      501|       4|150.0|2024-09-02|          600.0|     600.0|
|            4|       101|      503|       3|300.0|2024-09-02|          900.0|     900.0|
|            5|       104|      504|       1|450.0|2024-09-03|          450.0|     450.0|
|            6|       102|      502|       2|250.0|2024-09-03|          500.0|     500.0|
|            7|       103|      503|       5|300.0|2024-09-04|         1500.0|    1500.0|
|            8|       104|      504|       1|450.0|2024-09-04|          450.0|     450.0|
|         

In [53]:
### **Additional Challenge (Optional):**
#1. **Identify Repeat Customers:**
df.groupBy("CustomerID").count().filter(col("count") > 1).show()

#2. **Calculate the Average Sale Price Per Product:**
df.groupBy("ProductID").avg("Price").alias("AvgPricePerProduct").show()



+----------+-----+
|CustomerID|count|
+----------+-----+
|       101|    3|
|       103|    2|
|       102|    2|
|       104|    2|
+----------+-----+

+---------+----------+
|ProductID|avg(Price)|
+---------+----------+
|      501|     150.0|
|      504|     450.0|
|      502|     250.0|
|      505|     550.0|
|      503|     300.0|
+---------+----------+

